In [5]:
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import time
import numpy as np

### WebScraping 

In [ ]:
scraper = cloudscraper.create_scraper()
headers = {

'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'

}
dados_rj = ['https://www.zapimoveis.com.br/aluguel/apartamentos/rj+rio-de-janeiro/?onde=%2CRio+de+Janeiro%2CRio+de+Janeiro%2C%2C%2C%2C%2Ccity%2CBR>Rio+de+Janeiro>NULL>Rio+de+Janeiro%2C-22.906847%2C-43.172897%2C&tipos=apartamento_residencial&pagina=1&transacao=aluguel'] + \
             ['https://www.zapimoveis.com.br/aluguel/apartamentos/rj+rio-de-janeiro/?onde=%2CRio+de+Janeiro%2CRio+de+Janeiro%2C%2C%2C%2C%2Ccity%2CBR>Rio+de+Janeiro>NULL>Rio+de+Janeiro%2C-22.906847%2C-43.172897%2C&tipos=apartamento_residencial&pagina={}&transacao=aluguel'.format(i) for i in range(2, 323)]



In [ ]:
data = []
for dados in dados_rj:
    response = scraper.get(dados, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.content, 'html.parser')

    pagina_atual = soup.find_all('li', {'data-cy' : 'rp-property-cd'})
    
    for item in pagina_atual:
        local_element = item.find('h2', {'data-cy': 'rp-cardProperty-location-txt'})
        local = local_element.text.strip() if local_element else 'NA'

        area_element = item.find('li', {'data-cy': 'rp-cardProperty-propertyArea-txt'})
        area = area_element.text.strip() if area_element else 'NA'
        
        quartos_element = item.find('li', {'data-cy': 'rp-cardProperty-bedroomQuantity-txt'})
        quartos = quartos_element.text.strip() if quartos_element else 'NA'
        
        banheiros_element = item.find('li', {'data-cy': 'rp-cardProperty-bathroomQuantity-txt'})
        banheiros = banheiros_element.text.strip() if banheiros_element else 'NA'
        
        vagas_element = item.find('li', {'data-cy': 'rp-cardProperty-parkingSpacesQuantity-txt'})
        vagas = vagas_element.text.strip() if vagas_element else 'NA' 

        preco_element = item.find('div', {'data-cy' : 'rp-cardProperty-price-txt'})
        preco = preco_element.text.strip() if preco_element else 'NA'
        
        link_element = item.find('a', href=True)
        link = None
        if link_element and link_element['href'].startswith('https://www.zapimoveis.com.br/imovel/'):
            link = link_element['href']

        data.append({
            'Local': local, 
            'Preço': preco,
            'Área': area,
            'Quartos': quartos,
            'Banheiros': banheiros,
            'Vagas': vagas,
            'Link': link
        })
    
    time.sleep(1) 

In [ ]:

database = pd.DataFrame(data)
database.to_csv('imoveis_rio.csv', index = False)

### Ajustando a base de dados

In [59]:
db = pd.read_csv('imoveis_rio.csv', sep=',')
db['Bairro'] = 0
db['Aluguel(R$)'] = 0
db['Condomínio(R$)'] = 0
db['IPTU(R$)'] = 0
db['Área(m²)'] = db['Área']


In [60]:
bairros = db.groupby('Bairro').count()
bairro_zona_map = {
    'Leblon': 'Zona Sul', 'Ipanema': 'Zona Sul', 'Copacabana': 'Zona Sul', 'Botafogo': 'Zona Sul',
    'Jardim Botânico': 'Zona Sul', 'Flamengo': 'Zona Sul', 'Lagoa': 'Zona Sul', 'Laranjeiras': 'Zona Sul',
    'Humaitá': 'Zona Sul', 'Leme': 'Zona Sul', 'Cosme Velho': 'Zona Sul', 'Catete': 'Zona Sul',
    'Urca': 'Zona Sul', 'Joá': 'Zona Sul', 'São Conrado': 'Zona Sul', 'Gávea': 'Zona Sul',
    'Vidigal': 'Zona Sul', 

    'Barra da Tijuca': 'Zona Oeste', 'Barra Olímpica': 'Zona Oeste', 'Freguesia (Jacarepaguá)': 'Zona Oeste',
    'Jacarepaguá': 'Zona Oeste', 'Vargem Grande': 'Zona Oeste', 'Vargem Pequena': 'Zona Oeste',
    'Recreio dos Bandeirantes': 'Zona Oeste', 'Taquara': 'Zona Oeste', 'Anil': 'Zona Oeste',
    'Curicica': 'Zona Oeste', 'Gardênia Azul': 'Zona Oeste', 'Tanque': 'Zona Oeste',
    'Guaratiba': 'Zona Oeste', 'Barra de Guaratiba': 'Zona Oeste', 'Pedra de Guaratiba': 'Zona Oeste',
    'Campo Grande': 'Zona Oeste', 'Paciência': 'Zona Oeste', 'Senador Vasconcelos': 'Zona Oeste',
    'Bangu': 'Zona Oeste', 'Santa Cruz': 'Zona Oeste', 'Magalhães Bastos': 'Zona Oeste',
    'Jardim Sulacap': 'Zona Oeste', 'Realengo': 'Zona Oeste', 'Padre Miguel': 'Zona Oeste',
    'Inhoaíba': 'Zona Oeste', 'Camorim': 'Zona Oeste', 'Cidade de Deus': 'Zona Oeste',
    'Campo dos Afonsos': 'Zona Oeste', 'Santíssimo': 'Zona Oeste', 'Cosmos': 'Zona Oeste',
    'Senador Camará': 'Zona Oeste', 'Itanhangá': 'Zona Oeste', 'Pechincha': 'Zona Oeste', 'Vila Valqueire' : 'Zona Oeste', 'Praça Seca': 'Zona Oeste', 

    'Tijuca': 'Zona Norte', 'Vila Isabel': 'Zona Norte', 'Andaraí': 'Zona Norte', 'Grajaú': 'Zona Norte',
    'Maracanã': 'Zona Norte', 'Méier': 'Zona Norte', 'Encantado': 'Zona Norte', 'Cachambi': 'Zona Norte',
    'Engenho Novo': 'Zona Norte', 'Piedade': 'Zona Norte', 'Todos os Santos': 'Zona Norte',
    'Pilares': 'Zona Norte', 'Irajá': 'Zona Norte', 'Riachuelo': 'Zona Norte',
    'Alto da Boa Vista': 'Zona Norte', 'Engenho de Dentro': 'Zona Norte',
    'Quintino Bocaiúva': 'Zona Norte', 'Engenho da Rainha': 'Zona Norte',
    'Colégio': 'Zona Norte', 'Lins de Vasconcelos': 'Zona Norte', 'Tomás Coelho': 'Zona Norte',
    'Madureira': 'Zona Norte', 'Cascadura': 'Zona Norte', 'Vila da Penha': 'Zona Norte',
    'Del Castilho': 'Zona Norte', 'Coelho Neto': 'Zona Norte', 'Pavuna': 'Zona Norte',
    'Honório Gurgel': 'Zona Norte', 'Vila Kosmos': 'Zona Norte', 'Ramos': 'Zona Norte',
    'Cacuia': 'Zona Norte', 'Praça da Bandeira': 'Zona Norte', 'Brás de Pina': 'Zona Norte',
    'Praia da Bandeira': 'Zona Norte', 'Jardim Guanabara': 'Zona Norte',
    'Penha Circular': 'Zona Norte', 'Maria da Graça': 'Zona Norte', 'Jacaré': 'Zona Norte',
    'Jardim Carioca': 'Zona Norte', 'Campinho': 'Zona Norte', 'Bancários': 'Zona Norte',
    'Ribeira': 'Zona Norte', 'Cocotá': 'Zona Norte', 'Turiaçu': 'Zona Norte',
    'Vaz Lobo': 'Zona Norte', 'Tauá': 'Zona Norte', 'Sampaio': 'Zona Norte',
    'Engenheiro Leal': 'Zona Norte', 'Bento Ribeiro': 'Zona Norte', 'Rocha Miranda': 'Zona Norte',
    'Rocha': 'Zona Norte', 'Penha': 'Zona Norte', 'Braz de Pina': 'Zona Norte',
    'Vicente de Carvalho': 'Zona Norte', 'Higienópolis': 'Zona Norte', 'Água Santa': 'Zona Norte',
    'Oswaldo Cruz': 'Zona Norte', 'Inhaúma': 'Zona Norte', 'Parada de Lucas': 'Zona Norte',
    'Vasco da Gama': 'Zona Norte', 'Anchieta': 'Zona Norte', 'Portuguesa': 'Zona Norte',
    'Olaria': 'Zona Norte', 'Abolição': 'Zona Norte', 'Benfica': 'Zona Norte',
    'São Francisco Xavier': 'Zona Norte', 'Jardim América': 'Zona Norte',  'São Cristóvão': 'Zona Norte',
    'Guadalupe': 'Zona Norte', 'Ricardo de Albuquerque': 'Zona Norte', 'Cavalcanti': 'Zona Norte',
    'Estácio': 'Zona Norte', 'Mangueira': 'Zona Norte', 'Parque Anchieta': 'Zona Norte',
    'Marechal Hermes': 'Zona Norte', 'Zumbi': 'Zona Norte', 'Parque Colúmbia': 'Zona Norte',
    'Paquetá': 'Zona Norte', 'Vista Alegre': 'Zona Norte', 'Vigário Geral': 'Zona Norte', 'Cordovil': 'Zona Norte',
    'Freguesia (Ilha do Governador)': 'Zona Norte', 'Moneró': 'Zona Norte', 'Freguesia(Ilha do Governador)': 'Zona Norte', 'Bonsucesso': 'Zona Norte',

    'Centro': 'Zona Central', 'Rio Comprido': 'Zona Central',
    'Santo Cristo': 'Zona Central', 'Cidade Nova': 'Zona Central', 'Saúde': 'Zona Central',
    'Catumbi': 'Zona Central', 'Glória': 'Zona Central', 'Santa Teresa': 'Zona Central'
}


In [61]:
db['Bairro'] = db['Local'].str.extract(r'em\s+([^,]+),')

db['Aluguel(R$)'] = db['Preço'].str.extract(r'R\$\s*([\d\.]+(?:\/(?:mês|dia))?)')
db['IPTU(R$)'] = db['Preço'].str.extract(r'IPTU\s*R\$\s*([\d\.]+)')
db['Condomínio(R$)'] = db['Preço'].str.extract(r'Cond\.\s*R\$\s*([\d\.]+)')

db['Área(m²)'] = db['Área(m²)'].str.split()
db['Área(m²)'] = db['Área(m²)'].str[-2]

db['Quartos'] = db['Quartos'].str.split()
db['Quartos'] = db['Quartos'].str[-1]

db['Banheiros'] = db['Banheiros'].str.split()
db['Banheiros'] = db['Banheiros'].str[-1]

db['Vagas'] = db['Vagas'].str.split()
db['Vagas'] = db['Vagas'].str[-1]

db['Zona'] = db['Bairro'].apply(lambda x: bairro_zona_map.get(x, 'Outra'))


In [62]:
db.drop(columns=['Local', 'Preço'], inplace=True)
ordem = ['Bairro','Aluguel(R$)', 'Condomínio(R$)', 'IPTU(R$)', 'Área(m²)', 'Quartos', 'Banheiros', 'Vagas',  'Zona', 'Link']
db = db.reindex(columns=ordem)
db.head()

,Bairro,Aluguel(R$),Condomínio(R$),IPTU(R$),Área(m²),Quartos,Banheiros,Vagas,Zona,Link
0,Barra da Tijuca,9.650/mês,7.322,18.972,99,2,3,1,Zona Oeste,https://www.zapimoveis.com.br/imovel/venda-apa...
1,Leblon,9.000/mês,2.250,815,94,2,3,1,Zona Sul,https://www.zapimoveis.com.br/imovel/aluguel-a...
2,Ipanema,40.000/mês,4.438,3.711,280,4,1,3,Zona Sul,https://www.zapimoveis.com.br/imovel/aluguel-a...
3,Barra da Tijuca,4.500/mês,1.478,299,61,2,2,1,Zona Oeste,https://www.zapimoveis.com.br/imovel/aluguel-a...
4,Copacabana,14.000/mês,3.200,1.400,298-300,4,2-4,1-4,Zona Sul,NaN
